<a href="https://colab.research.google.com/github/martindevoto/machine-learning-notebooks-personal/blob/main/Intro_Snorkel_Labelling_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Uncomment line below to install snorkel the first time
!pip install --quiet snorkel

     |████████████████████████████████| 145 kB 7.7 MB/s 
     |████████████████████████████████| 1.7 MB 60.4 MB/s 
     |████████████████████████████████| 3.8 MB 62.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import re 

from sklearn.model_selection import train_test_split

In [ ]:
DISPLAY_ALL_TEXT = True

pd.set_option("display.max_colwidth", 0 if DISPLAY_ALL_TEXT else 50)

## 1. Loading Data

In [ ]:
df_filepath = r'/content/drive/MyDrive/smsspamcollection.tsv'

In [ ]:
df = pd.read_csv(df_filepath,sep='\t')

### Small EDA

In [ ]:
df.shape

(5572, 4)

In [ ]:
df.columns

Index(['label', 'message', 'length', 'punct'], dtype='object')

In [ ]:
df.rename(columns={'message': 'text'}, inplace=True)

In [ ]:
df = df[['text', 'label']]

In [ ]:
label_mapper = {'ham': 0,
                'spam': 1}

In [ ]:
df['label'] = df['label'].apply(lambda x: label_mapper[x])

In [ ]:
df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives around here though",0


In [ ]:
df.tail()

,text,label
5567,"This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.",1
5568,Will ü b going to esplanade fr home?,0
5569,"Pity, * was in mood for that. So...any other suggestions?",0
5570,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free,0
5571,Rofl. Its true to its name,0


In [ ]:
df[df['label'] == 0].sample(20, random_state=42)

,text,label
3714,If i not meeting ü all rite then i'll go home lor. If ü dun feel like comin it's ok.,0
1311,"I.ll always be there, even if its just in spirit. I.ll get a bb soon. Just trying to be sure i need it.",0
548,"Sorry that took so long, omw now",0
1324,I thk 50 shd be ok he said plus minus 10.. Did ü leave a line in between paragraphs?,0
3184,Dunno i juz askin cos i got a card got 20% off 4 a salon called hair sense so i tot it's da one ü cut ur hair.,0
1075,Aight ill get on fb in a couple minutes,0
1412,somewhere out there beneath the pale moon light someone think in of u some where out there where dreams come true... goodnite &amp; sweet dreams,0
3043,Slaaaaave ! Where are you ? Must I summon you to me all the time now ? Don't you wish to come to me on your own anymore?,0
14,I HAVE A DATE ON SUNDAY WITH WILL!!,0
5097,Sorry about that this is my mates phone and i didnt write it love Kate,0


In [ ]:
df[df['label'] == 1].sample(20, random_state=42)

,text,label
1456,Summers finally here! Fancy a chat or flirt with sexy singles in yr area? To get MATCHED up just reply SUMMER now. Free 2 Join. OptOut txt STOP Help08714742804,1
1853,"This is the 2nd time we have tried 2 contact u. U have won the 750 Pound prize. 2 claim is easy, call 08718726970 NOW! Only 10p per min. BT-national-rate",1
673,Get ur 1st RINGTONE FREE NOW! Reply to this msg with TONE. Gr8 TOP 20 tones to your phone every week just £1.50 per wk 2 opt out send STOP 08452810071 16,1
947,Ur cash-balance is currently 500 pounds - to maximize ur cash-in now send GO to 86688 only 150p/msg. CC: 08718720201 PO BOX 114/14 TCR/W1,1
2881,"Last Chance! Claim ur £150 worth of discount vouchers today! Text SHOP to 85023 now! SavaMob, offers mobile! T Cs SavaMob POBOX84, M263UZ. £3.00 Sub. 16",1
3642,"You can stop further club tones by replying ""STOP MIX"" See my-tone.com/enjoy. html for terms. Club tones cost GBP4.50/week. MFL, PO Box 1146 MK45 2WT (2/3)",1
1653,"For ur chance to win a £250 cash every wk TXT: ACTION to 80608. T's&C's www.movietrivia.tv custcare 08712405022, 1x150p/wk",1
1205,WIN a year supply of CDs 4 a store of ur choice worth £500 & enter our £100 Weekly draw txt MUSIC to 87066 Ts&Cs www.Ldew.com.subs16+1win150ppmx3,1
3968,YOU HAVE WON! As a valued Vodafone customer our computer has picked YOU to win a £150 prize. To collect is easy. Just call 09061743386,1
2691,sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p,1


In [ ]:
X, y = df[['text']], df['label']

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
df_train = X_train.merge(y_train, left_index=True, right_index=True,)

In [ ]:
df_test = X_test.merge(y_test, left_index=True, right_index=True,)

## Let's Snorkel

## 2. Writing Labeling Functions (LFs)

In [ ]:
# For clarity, we define constants to represent
# the class labels for spam, ham, and abstaining.
ABSTAIN = -1
HAM = 0
SPAM = 1

In [ ]:
from snorkel.labeling import labeling_function

In [ ]:
@labeling_function()
def pound(x):
    return SPAM if "£" in x.text.lower() else ABSTAIN

In [ ]:
@labeling_function()
def exclamation(x):
    return SPAM if "!" in x.text.lower() else ABSTAIN

In [ ]:
from snorkel.labeling import PandasLFApplier

In [ ]:
lfs = [pound, exclamation]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 3733/3733 [00:00<00:00, 30929.48it/s]


In [ ]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1, -1],
       [-1, -1]])

In [ ]:
coverage_pound, coverage_exclamation = (L_train != ABSTAIN).mean(axis=0)
print(f"pound coverage: {coverage_pound * 100:.1f}%")
print(f"exclamation coverage: {coverage_exclamation * 100:.1f}%")

pound coverage: 4.4%
exclamation coverage: 16.3%


### Snorkel offers several insights related to our labeling functions. Examples are:

* Polarity: The set of unique labels this LF outputs (excluding abstains)

* Coverage: The fraction of the dataset the LF labels

* Overlaps: The fraction of the dataset where this LF and at least one other LF label

* Conflicts: The fraction of the dataset where this LF and at least one other LF label and disagree

* Correct: The number of data points this LF labels correctly (if gold labels are provided)

* Incorrect: The number of data points this LF labels incorrectly (if gold labels are provided)

* Empirical Accuracy: The empirical accuracy of this LF (if gold labels are provided)

In [ ]:
from snorkel.labeling import LFAnalysis

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
pound,0,[1],0.043665,0.021966,0.0
exclamation,1,[1],0.163407,0.021966,0.0


In [ ]:
from snorkel.analysis import get_label_buckets

In [ ]:
buckets = get_label_buckets(L_train[:, 0], L_train[:, 1])
df_train.iloc[buckets[(ABSTAIN, SPAM)]].sample(10, random_state=1)

,text,label
1063,Ok. Not sure what time tho as not sure if can get to library before class. Will try. See you at some point! Have good eve.,0
4929,Just hopeing that wasn‘t too pissed up to remember and has gone off to his sisters or something!,0
5501,PRIVATE! Your 2003 Account Statement for 07808247860 shows 800 un-redeemed S. I. M. points. Call 08719899229 Identifier Code: 40411 Expires 06/11/04,1
4070,Hope youre not having too much fun without me!! see u tomorrow love jess x,0
3212,"OH YEAH,AND HAV A GREAT TIME IN NEWQUAY-SEND ME A POSTCARD !1 LOOK AFTER ALL THE GIRLS WHILE IM GONE(U KNOW THE 1IM TALKIN BOUT!)xx",0
2026,"Yes obviously, but you are the eggs-pert and the potato head… Speak soon!",0
4082,Hurry home. Soup is DONE!,0
793,Y?WHERE U AT DOGBREATH? ITS JUST SOUNDING LIKE JAN C THATS AL!!!!!!!!!,0
5116,Thanks. Fills me with complete calm and reassurance!,0
3473,"I think I‘m waiting for the same bus! Inform me when you get there, if you ever get there.",0


In [ ]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob

In [ ]:
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return HAM if x.polarity > 0.9 else ABSTAIN

In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return HAM if x.subjectivity >= 0.5 else ABSTAIN

In [ ]:
lfs = [textblob_polarity, textblob_subjectivity]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)

100%|██████████| 3733/3733 [00:03<00:00, 1096.68it/s]


In [ ]:
LFAnalysis(L_train, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
textblob_polarity,0,[0],0.00884,0.006965,0.0
textblob_subjectivity,1,[0],0.41977,0.006965,0.0


In [ ]:
@labeling_function()
def short_comment(x):
    """Ham comments are often short, such as 'cool video!'"""
    return HAM if len(x.text.split()) < 5 else ABSTAIN

In [ ]:
from snorkel.preprocess.nlp import SpacyPreprocessor

In [ ]:
# The SpacyPreprocessor parses the text in text_field and
# stores the new enriched representation in doc_field
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

In [ ]:
@labeling_function(pre=[spacy])
def has_person(x):
    """Ham comments mention specific people and are short."""
    if len(x.doc) < 20 and any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [ ]:
from snorkel.labeling.lf.nlp import nlp_labeling_function

In [ ]:
@nlp_labeling_function()
def has_person_nlp(x):
    """Ham comments mention specific people and are short."""
    if len(x.doc) < 20 and any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [ ]:
lfs = [
       pound,
       exclamation,
       textblob_polarity,
       textblob_subjectivity,
       short_comment,
       has_person,
       has_person_nlp
]

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|██████████| 1839/1839 [00:38<00:00, 47.94it/s]


In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
pound,0,[1],0.043665,0.030003,0.017948
exclamation,1,[1],0.163407,0.105277,0.093223
textblob_polarity,2,[0],0.008840,0.008304,0.005090
textblob_subjectivity,3,[0],0.419770,0.133941,0.096437
short_comment,4,[0],0.059737,0.023841,0.002143
has_person,5,[0],0.047415,0.047415,0.003482
has_person_nlp,6,[0],0.047415,0.047415,0.003482


## 3. Combining Labeling Function Outputs with the Label Model

In [ ]:
from snorkel.labeling.model import MajorityLabelVoter

In [ ]:
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [ ]:
from snorkel.labeling.model import LabelModel

In [ ]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [ ]:
majority_acc = majority_model.score(L=L_test, 
                                    Y=y_test.values,
                                    tie_break_policy="random")["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test,
                                    Y=y_test.values,
                                    tie_break_policy="random")["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   69.4%
Label Model Accuracy:     61.4%


## Filtering out unlabeled data points

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import probs_to_preds

In [ ]:
probs_train = label_model.predict_proba(L=L_train)

In [ ]:
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train)

In [ ]:
preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

In [ ]:
df_train.shape

(3733, 2)

In [ ]:
df_train_filtered.shape

(2143, 2)

## 4. Training a Classifier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 5))
X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [ ]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver="liblinear")
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(f"Test Accuracy: {sklearn_model.score(X=X_test, y=y_test) * 100:.1f}%")

Test Accuracy: 73.6%
